In [32]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

In [33]:
path_arquivos = 'arquivos'

In [63]:
df_2010 = pd.read_csv(f'{path_arquivos}/microdados_ed_basica_2010.csv', encoding='utf-8', sep=';')
df_2011 = pd.read_csv(f'{path_arquivos}/microdados_ed_basica_2011.csv', encoding='utf-8', sep=';')
df_2012 = pd.read_csv(f'{path_arquivos}/microdados_ed_basica_2012.csv', encoding='utf-8', sep=';')
df_2013 = pd.read_csv(f'{path_arquivos}/microdados_ed_basica_2013.csv', encoding='utf-8', sep=';')
df_2014 = pd.read_csv(f'{path_arquivos}/microdados_ed_basica_2014.csv', encoding='utf-8', sep=';')
df_2015 = pd.read_csv(f'{path_arquivos}/microdados_ed_basica_2015.csv', encoding='utf-8', sep=';')

df_final = pd.concat([df_2010, df_2011, df_2012, df_2013, df_2014, df_2015],
                     axis=0,
                     ignore_index=True)

C:\Users\GustavoValentim\AppData\Local\Temp\ipykernel_17128\92716148.py:1: DtypeWarning: Columns (20,21,25,27) have mixed types. Specify dtype option on import or set low_memory=False.
  df_2010 = pd.read_csv(f'{path_arquivos}/microdados_ed_basica_2010.csv', encoding='utf-8', sep=';')
C:\Users\GustavoValentim\AppData\Local\Temp\ipykernel_17128\92716148.py:2: DtypeWarning: Columns (20,21,27) have mixed types. Specify dtype option on import or set low_memory=False.
  df_2011 = pd.read_csv(f'{path_arquivos}/microdados_ed_basica_2011.csv', encoding='utf-8', sep=';')
C:\Users\GustavoValentim\AppData\Local\Temp\ipykernel_17128\92716148.py:3: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.
  df_2012 = pd.read_csv(f'{path_arquivos}/microdados_ed_basica_2012.csv', encoding='utf-8', sep=';')
C:\Users\GustavoValentim\AppData\Local\Temp\ipykernel_17128\92716148.py:4: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or 

In [64]:
# filtrando os dados do RJ e apenas colegios que possuem ensino médio

df_rj = df_final[(df_final['SG_UF'] == 'RJ') & (df_final['IN_MED'] == 1)]

dados_infraestrutura = [
    'NU_ANO_CENSO',
    'CO_ENTIDADE',
    'TP_DEPENDENCIA',
    'SG_UF',
    'IN_BIBLIOTECA_SALA_LEITURA',
    'IN_LABORATORIO_CIENCIAS',
    'IN_AUDITORIO',
    'IN_INTERNET',
]

df_rj = df_rj[dados_infraestrutura]

In [65]:
# carregando, filtrando e manipulando dados sobre o Enem por escola

df_enem = pd.read_csv(f'{path_arquivos}/MICRODADOS_ENEM_ESCOLA.csv',
                     encoding='utf-8',
                     sep=';')

years_to_include = [2010, 2011, 2012, 2013, 2014, 2015]
colunas_selecionadas = ['CO_ESCOLA_EDUCACENSO', 'NU_ANO', 'SG_UF_ESCOLA',
                       'TP_DEPENDENCIA_ADM_ESCOLA', 'NU_TAXA_PARTICIPACAO', 'NU_TAXA_APROVACAO', 'TP_LOCALIZACAO_ESCOLA', 
                       'NU_MEDIA_CN', 'NU_MEDIA_CH', 'NU_MEDIA_LP', 'NU_MEDIA_MT', 'NU_MEDIA_RED']

df_enem_final = df_enem[
    (df_enem['SG_UF_ESCOLA'] == 'RJ') &
    (df_enem['NU_ANO'].isin(years_to_include))
][colunas_selecionadas].copy()

df_enem_final = df_enem_final.rename(columns={
    'CO_ESCOLA_EDUCACENSO': 'CO_ENTIDADE',
    'NU_ANO': 'NU_ANO_CENSO',
    'SG_UF_ESCOLA': 'SG_UF',
    'TP_DEPENDENCIA_ADM_ESCOLA': 'TP_DEPENDENCIA'
})

# carregando, filtrando e manipulando dados sobre o INSE por escola

df_inse11a13 = pd.read_csv(f'{path_arquivos}/Indicador_INSE_por_Escola.csv',
                     encoding='utf-8',
                     sep=',')
df_inse15 = pd.read_csv(f'{path_arquivos}/INSE_2015.csv',
                     encoding='utf-8',
                     sep=',')

df_inse11a13 = df_inse11a13.rename(columns={
    'COD_ESCOLA': 'CO_ENTIDADE',
    'COD_ESTADO': 'CO_UF',
    'INSE - VALOR ABSOLUTO': 'INSE_VALOR_ABSOLUTO',
})

df_inse15 = df_inse15.rename(columns={
    'CO_ESCOLA': 'CO_ENTIDADE',
})

cols = ['CO_ENTIDADE', 'CO_UF', 'INSE_VALOR_ABSOLUTO']

print("Columns in df_inse11a13 after renaming:", df_inse11a13.columns.tolist())

df_inse11a13 = df_inse11a13[cols]
df_inse15 = df_inse15[cols]

df_inse = pd.concat([df_inse11a13, df_inse15],
                     axis=0,
                     ignore_index=True)

df_inse_final = df_inse[
    (df_inse['CO_UF'] == 33)
].copy()

df_inse_final = df_inse_final.drop(columns=['CO_UF'])

df_inse_final['INSE_VALOR_ABSOLUTO'] = df_inse_final['INSE_VALOR_ABSOLUTO'].astype(float)

inse_medio_df = df_inse_final.groupby('CO_ENTIDADE')['INSE_VALOR_ABSOLUTO'].mean().reset_index()
inse_medio_df.columns = ['CO_ENTIDADE', 'INSE_MEDIO']

inse_medio_df

C:\Users\GustavoValentim\AppData\Local\Temp\ipykernel_17128\2911634410.py:3: DtypeWarning: Columns (0,20) have mixed types. Specify dtype option on import or set low_memory=False.
  df_enem = pd.read_csv(f'{path_arquivos}/MICRODADOS_ENEM_ESCOLA.csv',


Columns in df_inse11a13 after renaming: ['CO_ENTIDADE', 'NOME_ESCOLA', 'CO_UF', 'NOME_ESTADO', 'COD_MUNICIPIO', 'NOME_MUNICIPIO', 'ID_AREA', 'AREA', 'ID_REDE', 'REDE', 'ID_LOCALIZACAO', 'LOCALIZACAO', 'QTD_ALUNOS_INSE', 'INSE_VALOR_ABSOLUTO', 'INSE - CLASSIFICAÇÃO']


,CO_ENTIDADE,INSE_MEDIO
0,33000026,47.785
1,33000042,47.420
2,33000077,50.615
3,33000115,47.510
4,33000158,50.415
...,...,...
5297,33548242,50.745
5298,33548250,58.750
5299,33551219,68.230
5300,33552231,62.600


In [66]:
notas_competencias = df_enem_final[['NU_MEDIA_CN', 'NU_MEDIA_CH', 'NU_MEDIA_LP', 'NU_MEDIA_MT', 'NU_MEDIA_RED']]

In [67]:
correlacao = notas_competencias.corr()
print(correlacao)

# boa correlação de pearson entre as competencias -> justificativa para unificar as pontuações em uma unica pontuação

              NU_MEDIA_CN  NU_MEDIA_CH  NU_MEDIA_LP  NU_MEDIA_MT  NU_MEDIA_RED
NU_MEDIA_CN      1.000000     0.885867     0.918387     0.915664      0.834201
NU_MEDIA_CH      0.885867     1.000000     0.845764     0.740300      0.784604
NU_MEDIA_LP      0.918387     0.845764     1.000000     0.859763      0.837568
NU_MEDIA_MT      0.915664     0.740300     0.859763     1.000000      0.793376
NU_MEDIA_RED     0.834201     0.784604     0.837568     0.793376      1.000000


In [68]:
scaler = StandardScaler()
notas_normalizadas = scaler.fit_transform(notas_competencias)

# normalização das notas por competência -> necessário por não possuirem mesmo range de nota

In [69]:
df_enem_final["NOTA_GERAL_ESCOLA"] = notas_normalizadas.mean(axis=1)

# média das competências pós normalização

In [70]:
df_encoded = pd.get_dummies(
    df_enem_final,
    columns=['TP_DEPENDENCIA', 'TP_LOCALIZACAO_ESCOLA'],
    dtype=int  # Garante que as colunas sejam 0 ou 1 (em vez de True/False)
)

# Visualize as novas colunas
print(df_encoded.filter(like='TP_').head())

# one hot encoding

        TP_DEPENDENCIA_1.0  TP_DEPENDENCIA_2.0  TP_DEPENDENCIA_3.0  \
114222                   0                   1                   0   
114223                   0                   1                   0   
114225                   0                   1                   0   
114226                   0                   1                   0   
114227                   0                   1                   0   

        TP_DEPENDENCIA_4.0  TP_LOCALIZACAO_ESCOLA_1.0  \
114222                   0                          1   
114223                   0                          1   
114225                   0                          1   
114226                   0                          1   
114227                   0                          1   

        TP_LOCALIZACAO_ESCOLA_2.0  
114222                          0  
114223                          0  
114225                          0  
114226                          0  
114227                          0  


In [71]:
df_encoded

,CO_ENTIDADE,NU_ANO_CENSO,SG_UF,NU_TAXA_PARTICIPACAO,NU_TAXA_APROVACAO,NU_MEDIA_CN,NU_MEDIA_CH,NU_MEDIA_LP,NU_MEDIA_MT,NU_MEDIA_RED,NOTA_GERAL_ESCOLA,TP_DEPENDENCIA_1.0,TP_DEPENDENCIA_2.0,TP_DEPENDENCIA_3.0,TP_DEPENDENCIA_4.0,TP_LOCALIZACAO_ESCOLA_1.0,TP_LOCALIZACAO_ESCOLA_2.0
114222,33000026.0,2011,RJ,56.52,79.4,430.27,425.30,495.71,445.68,346.15,-1.777510,0,1,0,0,1,0
114223,33000026.0,2012,RJ,65.00,96.1,453.97,488.86,470.10,477.81,396.92,-1.357614,0,1,0,0,1,0
114225,33000042.0,2010,RJ,33.00,77.0,449.63,501.99,454.93,458.54,554.69,-1.052234,0,1,0,0,1,0
114226,33000042.0,2013,RJ,57.65,91.1,442.61,482.44,455.54,485.18,424.08,-1.405972,0,1,0,0,1,0
114227,33000042.0,2014,RJ,50.00,93.7,465.30,531.72,495.83,457.50,412.65,-1.046283,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122617,33552231.0,2014,RJ,100.00,97.3,584.41,635.73,573.81,577.06,673.00,1.084774,0,0,0,1,1,0
122618,33552231.0,2015,RJ,100.00,98.9,578.66,643.76,583.45,607.97,750.00,1.402079,0,0,0,1,1,0
122619,33558230.0,2013,RJ,50.81,70.3,441.43,473.67,471.36,483.36,481.78,-1.235277,0,1,0,0,1,0
122620,33558230.0,2014,RJ,53.36,63.6,472.13,532.78,499.61,447.28,492.54,-0.828291,0,1,0,0,1,0


In [72]:
# Filtre apenas colunas numéricas
colunas_numericas = df_encoded.select_dtypes(include=['float64', 'int64']).columns
df_numerico = df_encoded[colunas_numericas]

# Agora calcule a correlação
correlacao = df_numerico.corr()
print(correlacao["NOTA_GERAL_ESCOLA"].sort_values(ascending=False))

# verificação da correlação entre a nota geral do ENEM por escola e as outras variáveis

NOTA_GERAL_ESCOLA            1.000000
NU_MEDIA_CN                  0.974690
NU_MEDIA_LP                  0.954800
NU_MEDIA_MT                  0.922165
NU_MEDIA_CH                  0.910843
NU_MEDIA_RED                 0.909619
NU_TAXA_PARTICIPACAO         0.616848
TP_DEPENDENCIA_4.0           0.614424
NU_TAXA_APROVACAO            0.379604
CO_ENTIDADE                  0.226703
TP_DEPENDENCIA_1.0           0.199436
TP_LOCALIZACAO_ESCOLA_1.0    0.116996
TP_DEPENDENCIA_3.0          -0.018612
TP_LOCALIZACAO_ESCOLA_2.0   -0.116996
TP_DEPENDENCIA_2.0          -0.671503
Name: NOTA_GERAL_ESCOLA, dtype: float64


In [73]:
# Mesclando o dataset do censo com o dataset do enem por escola

df_enem_escola = pd.merge(
    df_rj,
    df_encoded,
    how='inner',
    left_on=['CO_ENTIDADE', 'NU_ANO_CENSO'],
    right_on=['CO_ENTIDADE', 'NU_ANO_CENSO']
)

df_completo = pd.merge(
    df_enem_escola,
    inse_medio_df,
    how='inner',
    on=['CO_ENTIDADE']
)

In [74]:
df_completo.columns

Index(['NU_ANO_CENSO', 'CO_ENTIDADE', 'TP_DEPENDENCIA', 'SG_UF_x',
       'IN_BIBLIOTECA_SALA_LEITURA', 'IN_LABORATORIO_CIENCIAS', 'IN_AUDITORIO',
       'IN_INTERNET', 'SG_UF_y', 'NU_TAXA_PARTICIPACAO', 'NU_TAXA_APROVACAO',
       'NU_MEDIA_CN', 'NU_MEDIA_CH', 'NU_MEDIA_LP', 'NU_MEDIA_MT',
       'NU_MEDIA_RED', 'NOTA_GERAL_ESCOLA', 'TP_DEPENDENCIA_1.0',
       'TP_DEPENDENCIA_2.0', 'TP_DEPENDENCIA_3.0', 'TP_DEPENDENCIA_4.0',
       'TP_LOCALIZACAO_ESCOLA_1.0', 'TP_LOCALIZACAO_ESCOLA_2.0', 'INSE_MEDIO'],
      dtype='object')

In [75]:
columnsToRemove = [
    'NU_ANO_CENSO',
    'SG_UF_x',
    'CO_ENTIDADE',
    'TP_DEPENDENCIA',
    'SG_UF_y',
    'NU_MEDIA_CN', 'NU_MEDIA_CH', 'NU_MEDIA_LP', 'NU_MEDIA_MT', 'NU_MEDIA_RED'
  ]
df_completo = df_completo.drop(columns=columnsToRemove)

df = df_completo.dropna()

In [76]:
df_completo

,IN_BIBLIOTECA_SALA_LEITURA,IN_LABORATORIO_CIENCIAS,IN_AUDITORIO,IN_INTERNET,NU_TAXA_PARTICIPACAO,NU_TAXA_APROVACAO,NOTA_GERAL_ESCOLA,TP_DEPENDENCIA_1.0,TP_DEPENDENCIA_2.0,TP_DEPENDENCIA_3.0,TP_DEPENDENCIA_4.0,TP_LOCALIZACAO_ESCOLA_1.0,TP_LOCALIZACAO_ESCOLA_2.0,INSE_MEDIO
0,1,1,0,1,24.20,65.8,-0.610938,0,1,0,0,1,0,51.675
1,1,1,0,1,29.60,75.2,-0.466760,0,1,0,0,1,0,49.830
2,1,1,0,1,92.90,86.2,1.483973,0,0,0,1,1,0,63.020
3,1,1,0,1,54.50,99.0,2.297554,1,0,0,0,1,0,62.080
4,1,0,0,1,91.70,91.1,1.662387,0,0,0,1,1,0,59.070
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7029,1,1,1,1,88.32,89.4,-0.065845,0,1,0,0,1,0,53.230
7030,1,1,1,1,88.00,97.1,0.544343,0,0,0,1,1,0,62.360
7031,1,0,1,1,95.65,93.1,1.349667,0,0,0,1,1,0,62.675
7032,1,1,1,1,91.30,94.1,1.035387,0,0,0,1,1,0,64.420


In [77]:
df_completo.corr()

,IN_BIBLIOTECA_SALA_LEITURA,IN_LABORATORIO_CIENCIAS,IN_AUDITORIO,IN_INTERNET,NU_TAXA_PARTICIPACAO,NU_TAXA_APROVACAO,NOTA_GERAL_ESCOLA,TP_DEPENDENCIA_1.0,TP_DEPENDENCIA_2.0,TP_DEPENDENCIA_3.0,TP_DEPENDENCIA_4.0,TP_LOCALIZACAO_ESCOLA_1.0,TP_LOCALIZACAO_ESCOLA_2.0,INSE_MEDIO
IN_BIBLIOTECA_SALA_LEITURA,1.000000,0.214902,0.160736,0.048236,0.051542,0.059000,-0.014878,0.037658,0.037492,0.021095,-0.052432,0.002081,-0.002081,-0.027279
IN_LABORATORIO_CIENCIAS,0.214902,1.000000,0.228657,0.026527,0.119399,0.031227,0.200468,0.116844,-0.090438,-0.000835,0.057518,0.062626,-0.062626,0.191288
IN_AUDITORIO,0.160736,0.228657,1.000000,-0.000956,0.174059,0.088822,-0.040664,0.049628,0.082238,-0.026414,-0.089200,0.018233,-0.018233,-0.025527
IN_INTERNET,0.048236,0.026527,-0.000956,1.000000,0.029198,0.024487,0.055600,0.014772,-0.054762,-0.059593,0.063882,0.051012,-0.051012,0.059776
NU_TAXA_PARTICIPACAO,0.051542,0.119399,0.174059,0.029198,1.000000,0.568984,0.617548,0.098110,-0.607392,-0.014786,0.578409,0.060417,-0.060417,0.626729
NU_TAXA_APROVACAO,0.059000,0.031227,0.088822,0.024487,0.568984,1.000000,0.379701,0.036778,-0.481248,-0.030993,0.475353,-0.014415,0.014415,0.397890
NOTA_GERAL_ESCOLA,-0.014878,0.200468,-0.040664,0.055600,0.617548,0.379701,1.000000,0.199916,-0.673400,-0.018588,0.616222,0.117149,-0.117149,0.859862
TP_DEPENDENCIA_1.0,0.037658,0.116844,0.049628,0.014772,0.098110,0.036778,0.199916,1.000000,-0.119128,-0.016253,-0.154890,-0.002730,0.002730,0.056005
TP_DEPENDENCIA_2.0,0.037492,-0.090438,0.082238,-0.054762,-0.607392,-0.481248,-0.673400,-0.119128,1.000000,-0.098206,-0.935884,-0.134288,0.134288,-0.756003
TP_DEPENDENCIA_3.0,0.021095,-0.000835,-0.026414,-0.059593,-0.014786,-0.030993,-0.018588,-0.016253,-0.098206,1.000000,-0.127687,0.011246,-0.011246,-0.034043


In [78]:
print("Valores faltantes em y (NOTA_GERAL_ESCOLA):", df["NOTA_GERAL_ESCOLA"].isna().sum())
print("\nValores faltantes em X:")
print(X.isna().sum())

Valores faltantes em y (NOTA_GERAL_ESCOLA): 0

Valores faltantes em X:
TP_DEPENDENCIA               0
IN_LABORATORIO_CIENCIAS      0
NU_TAXA_PARTICIPACAO         0
NU_TAXA_APROVACAO            0
TP_DEPENDENCIA_2.0           0
TP_DEPENDENCIA_4.0           0
TP_LOCALIZACAO_ESCOLA_1.0    0
TP_LOCALIZACAO_ESCOLA_2.0    0
INSE_MEDIO                   0
dtype: int64


In [ ]:
# df_completo = df_completo.dropna(subset=['NOTA_GERAL_ESCOLA'])

In [ ]:
# Preenche com a mediana por grupo de TP_DEPENDENCIA
#df_completo['NU_TAXA_APROVACAO'] = df_completo.groupby('TP_DEPENDENCIA')['NU_TAXA_APROVACAO'].transform(
#    lambda x: x.fillna(x.median())
#)

In [ ]:
print("Valores faltantes restantes:")
print(df[['NU_TAXA_APROVACAO', 'NOTA_GERAL_ESCOLA']].isna().sum())

Valores faltantes restantes:
NU_TAXA_APROVACAO    295
NOTA_GERAL_ESCOLA      0
dtype: int64


In [82]:
# Lista de colunas a remover (atualizada)
cols_remover = [
    'NU_MEDIA_CN', 'NU_MEDIA_CH', 'NU_MEDIA_LP', 'NU_MEDIA_MT', 'NU_MEDIA_RED',
    'CO_ENTIDADE', 'SG_UF_x', 'SG_UF_y', 'IN_BIBLIOTECA_SALA_LEITURA', 
    'IN_AUDITORIO', 'IN_INTERNET', 'TP_DEPENDENCIA_3.0', 'TP_DEPENDENCIA_1.0',
    'NU_ANO_CENSO'  # Adicionei pois não é relevante para o modelo
]

# Garantir que todas as colunas em cols_remover existam no DataFrame
cols_remover = [col for col in cols_remover if col in df.columns]

# Preparar X e y
X = df.drop(columns=['NOTA_GERAL_ESCOLA'] + cols_remover)
y = df['NOTA_GERAL_ESCOLA']

# Verificar consistência
print(f"Tamanho de X: {X.shape}, Tamanho de y: {y.shape}")

# Treinar o modelo
model = RandomForestRegressor(random_state=42)
model.fit(X, y)

importancias = pd.Series(model.feature_importances_, index=X.columns).sort_values(ascending=False)
print("Importância das features:\n", importancias)

Tamanho de X: (6712, 8), Tamanho de y: (6712,)
Importância das features:
 INSE_MEDIO                   0.826867
NU_TAXA_PARTICIPACAO         0.084329
NU_TAXA_APROVACAO            0.061983
TP_DEPENDENCIA_4.0           0.013114
IN_LABORATORIO_CIENCIAS      0.008411
TP_DEPENDENCIA_2.0           0.003264
TP_LOCALIZACAO_ESCOLA_1.0    0.001024
TP_LOCALIZACAO_ESCOLA_2.0    0.001007
dtype: float64
